# **Please dont forget to Upvote if your learned something new :)**

**One Notebook: Feature Engineering-->Knn Folds-->Level0:-XGBoost|CatBoost|LightGBM-->Level1:-XgBoost|RandomForest:-Submission,csv :D**

Thakns to Luca Massaron for sharing stratify folds based ona k-means

In this competition there are a lot of discussions and notebooks on hyper-parameter tuning, XGBoost and stacking, since they are the most effective techniques.

Yet, something is missing. For instance, there little reasoning has been done on the best way to cross-validate.

Actually, if you watch carefully the data, it seems like data distributions are segregated into specific portions of space, something reminiscent ot me of the Madelon dataset created by Isabelle Guyon, one of the ideators of the Support Vector Machines (see for Guyon's contribution: https://www.kdnuggets.com/2016/07/guyon-data-mining-history-svm-support-vector-machines.html for the Madelon dataset see instead: https://archive.ics.uci.edu/ml/datasets/madelon).

I therefore tried to stratifiy my folds based on a k-means clustering of the non-noisy data (see https://www.kaggle.com/c/30-days-of-ml/discussion/267931) and my local cv has become more reliable (very correlated with the public leaderboard) and my models are performing much better with cv prediction.

Try it and let me know, if it works also on your models!

Happy Kaggling!

In [ ]:
# Importing core libraries
import numpy as np
import pandas as pd
import joblib

# Importing from Scikit-Learn
from sklearn.model_selection import StratifiedKFold, KFold
from sklearn.preprocessing import OrdinalEncoder
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA

import numpy as np
import pandas as pd
import scipy as sp

import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import OrdinalEncoder
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold, KFold
from sklearn.metrics import mean_squared_error
from mlxtend.preprocessing import minmax_scaling

from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor
import optuna
from sklearn import preprocessing
from sklearn.linear_model import LinearRegression
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA

import joblib
import os

In [ ]:
# Loading data 
X_train_full = pd.read_csv("../input/30-days-of-ml/train.csv")
X_test_full = pd.read_csv("../input/30-days-of-ml/test.csv")

In [ ]:
# Preparing data as a tabular matrix
X_train=X_train_full.copy()
X_test=X_test_full.copy()
y_train = X_train.target
X_train = X_train.set_index('id').drop('target', axis='columns')
X_test = X_test.set_index('id')

In [ ]:
# Pointing out categorical features
categoricals = [item for item in X_train.columns if 'cat' in item]

In [ ]:
# Dealing with categorical data using get_dummies
dummies = pd.get_dummies(X_train.append(X_test)[categoricals])
X_train[dummies.columns] = dummies.iloc[:len(X_train), :]
X_test[dummies.columns] = dummies.iloc[len(X_train): , :]
del(dummies)

In [ ]:
# Dealing with cuniquegorical data using OrdinalEncoder (only when there are 3 or more levels)
ordinal_encoder = OrdinalEncoder()
X_train[categoricals[3:]] = ordinal_encoder.fit_transform(X_train[categoricals[3:]]).astype(int)
X_test[categoricals[3:]] = ordinal_encoder.transform(X_test[categoricals[3:]]).astype(int)
X_train = X_train.drop(categoricals[:3], axis="columns")
X_test = X_test.drop(categoricals[:3], axis="columns")

In [ ]:
# Feature selection (https://www.kaggle.com/lucamassaron/tutorial-feature-selection-with-boruta-shap)
important_features = ['cat1_A', 'cat1_B', 'cat5', 'cat8', 'cat8_C', 'cat8_E', 'cont0', 
                      'cont1', 'cont10', 'cont11', 'cont12', 'cont13', 'cont2', 'cont3', 
                      'cont4', 'cont5', 'cont6', 'cont7', 'cont8', 'cont9']

categoricals = ['cat5', 'cat8']

X_train = X_train[important_features]
X_test = X_test[important_features]

In [ ]:
# Stratifying the data

pca = PCA(n_components=16, random_state=0)
km = KMeans(n_clusters=32, random_state=0)

pca.fit(X_train)
km.fit(pca.transform(X_train))

print(np.unique(km.labels_, return_counts=True))

y_stratified = km.labels_

In [ ]:
# Creating your folds for repeated use (for instance, stacking)
#To save some time, I am using already saved Knn fold
"""
folds = 10
#seeds = [42, 0, 101]
seeds = [42]
fold_idxs = list()

for seed in seeds:
    skf = StratifiedKFold(n_splits=folds,
                          shuffle=True, 
                          random_state=seed)

    fold_idxs.append(list(skf.split(X_train, y_stratified)))
"""

In [ ]:
"""
# Checking the produced folds
for j, fold_idxs_ in enumerate(fold_idxs):
    print(f"\n--- seed={seeds[j]} ---")
    for k, (train_idx, validation_idx) in enumerate(fold_idxs_):
        print(f"fold {k} train idxs: {len(train_idx)} validation idxs: {len(validation_idx)} -> {validation_idx[:10]}")
"""        

In [ ]:
#Loading existing already created Fold
sample_submission = pd.read_csv("../input/30-days-of-ml/sample_submission.csv")
# Storing into the notebook for future use
#joblib.dump(fold_idxs, './fold_idxs.job')
# Retrieving from the notebook
seeds = [42]
fold_idxs = joblib.load('../input/30daysfoldknn/fold_idxs.job')

***#XGboost to create Level0 prediction
#Parameteres are already tuned with Optuna***

In [ ]:
"""
# Checking the produced folds
final_test_predictions = []
final_valid_predictions = {}
scores = []
for j, fold_idxs_ in enumerate(fold_idxs):
    print(f"\n--- seed={seeds[j]} ---")
    for k, (train_idx, validation_idx) in enumerate(fold_idxs_):
        print(f"fold {k} train idxs: {len(train_idx)} validation idxs: {len(validation_idx)} -> {validation_idx[:10]}")
        fold=k
        xtrain =  X_train.iloc[train_idx].reset_index(drop=True)
        xvalid = X_train.iloc[validation_idx].reset_index(drop=True)
        xtest = X_test.copy()
        valid_ids = X_train_full.iloc[validation_idx]['id']
        
        ytrain = X_train_full.iloc[train_idx].target
        yvalid = X_train_full.iloc[validation_idx].target
    
        params = {'learning_rate': 0.03097703420625469, 'reg_lambda': 0.03821095322549157, 
              'reg_alpha': 23.13181079976304, 'subsample': 0.9494442458700542, 
              'colsample_bytree': 0.11807135201147481, 'max_depth': 3,'gamma': 0, 'min_child_weight': 1}
    
    
        model = XGBRegressor(
            random_state=42,
            n_estimators=10000,
            **params
        )
        model.fit(xtrain, ytrain)
        preds_valid = model.predict(xvalid)
        test_preds = model.predict(xtest)
        final_test_predictions.append(test_preds)
        final_valid_predictions.update(dict(zip(valid_ids, preds_valid)))
        rmse = mean_squared_error(yvalid, preds_valid, squared=False)
        print(fold, rmse)
        scores.append(rmse)

print(np.mean(scores), np.std(scores))

final_valid_predictions = pd.DataFrame.from_dict(final_valid_predictions, orient="index").reset_index()
final_valid_predictions.columns = ["id", "pred_1"]
final_valid_predictions.to_csv("level0_train_pred_1.csv", index=False)

sample_submission.target = np.mean(np.column_stack(final_test_predictions), axis=1)
sample_submission.columns = ["id", "pred_1"]
sample_submission.to_csv("level0_test_pred_1.csv", index=False)
"""

**#Catboost to create Level0 prediction
#Parameteres are already tuned with Optuna**

In [ ]:
"""
# Checking the produced folds
final_test_predictions = []
final_valid_predictions = {}
scores = []
for j, fold_idxs_ in enumerate(fold_idxs):
    print(f"\n--- seed={seeds[j]} ---")
    for k, (train_idx, validation_idx) in enumerate(fold_idxs_):
        print(f"fold {k} train idxs: {len(train_idx)} validation idxs: {len(validation_idx)} -> {validation_idx[:10]}")
        fold=k
        xtrain =  X_train.iloc[train_idx].reset_index(drop=True)
        xvalid = X_train.iloc[validation_idx].reset_index(drop=True)
        xtest = X_test.copy()
        valid_ids = X_train_full.iloc[validation_idx]['id']
        
        ytrain = X_train_full.iloc[train_idx].target
        yvalid = X_train_full.iloc[validation_idx].target
    
        params = {'random_state':2020,'learning_rate': 0.09371412415401288, 'l2_leaf_reg': 5.708567720284058, 
              'max_bin': 355, 'subsample': 0.7102809231581142, 
              'min_data_in_leaf': 165, 'max_depth': 7}
    
        model = CatBoostRegressor(
            n_estimators=25000,
            **params
        )
        model.fit(xtrain, ytrain)
        preds_valid = model.predict(xvalid)
        test_preds = model.predict(xtest)
        final_test_predictions.append(test_preds)
        final_valid_predictions.update(dict(zip(valid_ids, preds_valid)))
        rmse = mean_squared_error(yvalid, preds_valid, squared=False)
        print(fold, rmse)
        scores.append(rmse)

print(np.mean(scores), np.std(scores))

final_valid_predictions = pd.DataFrame.from_dict(final_valid_predictions, orient="index").reset_index()
final_valid_predictions.columns = ["id", "pred_2"]
final_valid_predictions.to_csv("level0_train_pred_2.csv", index=False)

sample_submission.target = np.mean(np.column_stack(final_test_predictions), axis=1)
sample_submission.columns = ["id", "pred_2"]
sample_submission.to_csv("level0_test_pred_2.csv", index=False)
"""

**#LightGBM to create Level0 prediction**

In [ ]:
"""
# Checking the produced folds
final_test_predictions = []
final_valid_predictions = {}
scores = []
for j, fold_idxs_ in enumerate(fold_idxs):
    print(f"\n--- seed={seeds[j]} ---")
    for k, (train_idx, validation_idx) in enumerate(fold_idxs_):
        print(f"fold {k} train idxs: {len(train_idx)} validation idxs: {len(validation_idx)} -> {validation_idx[:10]}")
        fold=k
        xtrain =  X_train.iloc[train_idx].reset_index(drop=True)
        xvalid = X_train.iloc[validation_idx].reset_index(drop=True)
        xtest = X_test.copy()
        valid_ids = X_train_full.iloc[validation_idx]['id']
        
        ytrain = X_train_full.iloc[train_idx].target
        yvalid = X_train_full.iloc[validation_idx].target
    
        params = {'metric': 'RMSE','feature_pre_filter': False,
              'lambda_l1': 0.45,
              'lambda_l2': 4.8,
              'learning_rate': 0.008,
              'num_trees': 50000,
              'num_leaves': 10, 
              'feature_fraction': 0.4, 
              'bagging_fraction': 1.0, 
              'bagging_freq': 0, 
              'min_child_samples': 100,
              'num_threads': 4}
    
    
        model = LGBMRegressor(**params)
        model.fit(xtrain, ytrain)
        preds_valid = model.predict(xvalid)
        test_preds = model.predict(xtest)
        final_test_predictions.append(test_preds)
        final_valid_predictions.update(dict(zip(valid_ids, preds_valid)))
        rmse = mean_squared_error(yvalid, preds_valid, squared=False)
        print(fold, rmse)
        scores.append(rmse)

print(np.mean(scores), np.std(scores))

final_valid_predictions = pd.DataFrame.from_dict(final_valid_predictions, orient="index").reset_index()
final_valid_predictions.columns = ["id", "pred_3"]
final_valid_predictions.to_csv("level0_train_pred_3.csv", index=False)

sample_submission.target = np.mean(np.column_stack(final_test_predictions), axis=1)
sample_submission.columns = ["id", "pred_3"]
sample_submission.to_csv("level0_test_pred_3.csv", index=False)
"""

**#Getting level 0 prediction across all 3 models and stacking-up**

In [ ]:
#df = pd.read_csv("../input/summary30daysfinalstacking/train_folds_10.csv")
#df_test = pd.read_csv("../input/30-days-of-ml/test.csv")
df=X_train.copy()
df_test=X_test.copy()
#df['id']=X_train.index
#df_test['id']=X_test.index

sample_submission = pd.read_csv("../input/30-days-of-ml/sample_submission.csv")

df1 = pd.read_csv("../input/30daysfoldknn/level0_train_pred_1.csv")
df2 = pd.read_csv("../input/30daysfoldknn/level0_train_pred_2.csv")
df3 = pd.read_csv("../input/30daysfoldknn/level0_train_pred_3.csv")

df_test1 = pd.read_csv("../input/30daysfoldknn/level0_test_pred_1.csv")
df_test2 = pd.read_csv("../input/30daysfoldknn/level0_test_pred_2.csv")
df_test3 = pd.read_csv("../input/30daysfoldknn/level0_test_pred_3.csv")

df = df.merge(df1, on="id", how="left")
df = df.merge(df2, on="id", how="left")
df = df.merge(df3, on="id", how="left")

df_test = df_test.merge(df_test1, on="id", how="left")
df_test = df_test.merge(df_test2, on="id", how="left")
df_test = df_test.merge(df_test3, on="id", how="left")

**#Using level 0 prediction to create level 1 prediction on XGBoost**

In [ ]:
useful_features = ["pred_1", "pred_2","pred_3"]
df_test = df_test[useful_features]

final_test_predictions = []
final_valid_predictions = {}
scores = []
for j, fold_idxs_ in enumerate(fold_idxs):
    print(f"\n--- seed={seeds[j]} ---")
    for k, (train_idx, validation_idx) in enumerate(fold_idxs_):
        print(f"fold {k} train idxs: {len(train_idx)} validation idxs: {len(validation_idx)} -> {validation_idx[:10]}")
        fold=k
        xtrain =  df.iloc[train_idx].reset_index(drop=True)
        xvalid = df.iloc[validation_idx].reset_index(drop=True)
        xtest = df_test.copy()
        valid_ids = xvalid.id.values.tolist()
        
        ytrain = X_train_full.iloc[train_idx].target
        yvalid = X_train_full.iloc[validation_idx].target
    
        xtrain = xtrain[useful_features]
        xvalid = xvalid[useful_features]

        params = {
        'random_state': 1, 
        'booster': 'gbtree',
        'n_estimators': 7000,
        'learning_rate': 0.03,
        'max_depth': 2
        }
    
        model = XGBRegressor(
        n_jobs=4,
        **params
        )
        model.fit(xtrain, ytrain, early_stopping_rounds=300, eval_set=[(xvalid, yvalid)], verbose=1000)
        preds_valid = model.predict(xvalid)
        test_preds = model.predict(xtest)
        final_test_predictions.append(test_preds)
        final_valid_predictions.update(dict(zip(valid_ids, preds_valid)))
        rmse = mean_squared_error(yvalid, preds_valid, squared=False)
        print(fold, rmse)
        scores.append(rmse)

print(np.mean(scores), np.std(scores))
final_valid_predictions = pd.DataFrame.from_dict(final_valid_predictions, orient="index").reset_index()
final_valid_predictions.columns = ["id", "pred_1"]
final_valid_predictions.to_csv("level1_train_pred_1.csv", index=False)

sample_submission.target = np.mean(np.column_stack(final_test_predictions), axis=1)
sample_submission.columns = ["id", "pred_1"]
sample_submission.to_csv("level1_test_pred_1.csv", index=False)

**#Using level 0 prediction to create level 1 prediction on Random Forest**

In [ ]:
sample_submission = pd.read_csv("../input/30-days-of-ml/sample_submission.csv")
useful_features = ["pred_1", "pred_2","pred_3"]
df_test = df_test[useful_features]

final_test_predictions = []
final_valid_predictions = {}
scores = []
for j, fold_idxs_ in enumerate(fold_idxs):
    print(f"\n--- seed={seeds[j]} ---")
    for k, (train_idx, validation_idx) in enumerate(fold_idxs_):
        print(f"fold {k} train idxs: {len(train_idx)} validation idxs: {len(validation_idx)} -> {validation_idx[:10]}")
        fold=k
        xtrain =  df.iloc[train_idx].reset_index(drop=True)
        xvalid = df.iloc[validation_idx].reset_index(drop=True)
        xtest = df_test.copy()
        valid_ids = xvalid.id.values.tolist()
        
        ytrain = X_train_full.iloc[train_idx].target
        yvalid = X_train_full.iloc[validation_idx].target
    
        xtrain = xtrain[useful_features]
        xvalid = xvalid[useful_features]
    

        model = RandomForestRegressor(n_estimators=1000, n_jobs=-1, max_depth=3)
        
        model.fit(xtrain, ytrain)
        preds_valid = model.predict(xvalid)
        test_preds = model.predict(xtest)
        final_test_predictions.append(test_preds)
        final_valid_predictions.update(dict(zip(valid_ids, preds_valid)))
        rmse = mean_squared_error(yvalid, preds_valid, squared=False)
        print(fold, rmse)
        scores.append(rmse)

print(np.mean(scores), np.std(scores))
final_valid_predictions = pd.DataFrame.from_dict(final_valid_predictions, orient="index").reset_index()
final_valid_predictions.columns = ["id", "pred_2"]
final_valid_predictions.to_csv("level1_train_pred_2.csv", index=False)

sample_submission.target = np.mean(np.column_stack(final_test_predictions), axis=1)
sample_submission.columns = ["id", "pred_2"]
sample_submission.to_csv("level1_test_pred_2.csv", index=False)

In [ ]:
#Blending

df=X_train.copy()
df_test=X_test.copy()
#df['id']=X_train.index
#df_test['id']=X_test.index
sample_submission = pd.read_csv("../input/30-days-of-ml/sample_submission.csv")

#df = pd.read_csv("../input/summary30daysfinalstacking/train_folds_10.csv")
#df_test = pd.read_csv("../input/30-days-of-ml/test.csv")

df1 = pd.read_csv("level1_train_pred_1.csv")
df2 = pd.read_csv("level1_train_pred_2.csv")

df_test1 = pd.read_csv("level1_test_pred_1.csv")
df_test2 = pd.read_csv("level1_test_pred_2.csv")

df = df.merge(df1, on="id", how="left")
df = df.merge(df2, on="id", how="left")

df_test = df_test.merge(df_test1, on="id", how="left")
df_test = df_test.merge(df_test2, on="id", how="left")

df.head()

Level 1 prediction used to linearly map it to the target..LinearRegression

In [ ]:
sample_submission = pd.read_csv("../input/30-days-of-ml/sample_submission.csv")

useful_features = ["pred_1", "pred_2"]
df_test = df_test[useful_features]

final_predictions = []
scores = []

for j, fold_idxs_ in enumerate(fold_idxs):
    print(f"\n--- seed={seeds[j]} ---")
    for k, (train_idx, validation_idx) in enumerate(fold_idxs_):
        print(f"fold {k} train idxs: {len(train_idx)} validation idxs: {len(validation_idx)} -> {validation_idx[:10]}")
        fold=k
        xtrain =  X_train.iloc[train_idx].reset_index(drop=True)
        xvalid = X_train.iloc[validation_idx].reset_index(drop=True)
        xtest = X_test.copy()
        valid_ids = X_train_full.iloc[validation_idx]['id']
        
        ytrain = X_train_full.iloc[train_idx].target
        yvalid = X_train_full.iloc[validation_idx].target
    

        model = LinearRegression()
        model.fit(xtrain, ytrain)
        
        preds_valid = model.predict(xvalid)
        test_preds = model.predict(xtest)
        final_predictions.append(test_preds)
        rmse = mean_squared_error(yvalid, preds_valid, squared=False)
        print(fold, rmse)
        scores.append(rmse)

In [ ]:
sample_submission.target = np.mean(np.column_stack(final_predictions), axis=1)
sample_submission.to_csv("submission.csv", index=False)